<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-26 09:32:43
-------------------
qualified stocks: 94
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.99 L
Current:  1.29 C
-------------------
Today PnL: 72.34 K (0.56%)
Current PnL: -16.51 L (-11.71%)
CY Booked + Current PnL: -12.22 L (-8.66%)
-------------------
Total profit:  3.79 L
Total loss:  -20.30 L
-------------------
Total Booked + Current PnL: 14.76 L (12.94%)
Total Booked PnL: 31.27 L (27.42%)
Curr Year Booked PnL: 4.29 L (3.32%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 78.48 L (60.73%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.40 C (108.17%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBILIFE,1496.49,1924.99,1.70,H-LC,46.81,100710.0,16907.0,7090.0,-0.02,20.17,7.04,28.63,42.0,2.38,0.81,34.58,AR,ATH,INSURANCE
62,RELIANCE,1291.83,1526.00,5.36,X-LC,52.13,156666.0,15857.0,9666.0,0.74,11.26,6.17,18.13,17.0,1.64,1.26,23.70,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,91.49,100004.0,21642.0,12390.0,0.94,27.62,12.39,43.43,71.0,1.75,0.80,33.75,XY24,NTT,MISC
56,PIDILITIND,2504.06,3576.00,-17.01,X-LC,23.40,91953.0,16831.0,15329.0,1.10,22.41,16.67,42.81,11.0,1.10,0.74,15.32,X40,BTT,CHEMICALS
69,SBIN,760.30,863.00,-13.45,M-LC,13.83,211033.0,9554.0,17663.0,0.74,4.74,8.37,13.51,73.0,0.54,1.70,17.94,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VAIBHAVGBL,350.21,521.0,61.98,X-SC,77.66,108174.0,-43817.0,117942.0,3.07,-28.83,109.03,48.77,59.0,-0.37,0.87,24.25,XR,NTT,JEWELLERY
74,SONACOMS,578.05,1006.0,-25.14,M-MC,97.87,96792.0,-4367.0,79253.0,2.94,-4.32,81.88,74.03,151.0,-0.06,0.78,36.03,AR,BTT,AUTO
59,RAJESHEXPO,517.65,583.0,1848.23,M-SC,59.57,53154.0,-84023.0,101343.0,2.45,-61.25,190.66,12.62,208.0,-0.83,0.43,32.20,X40N,NTT,JEWELLERY
7,ASIANTILES,75.41,137.0,5638.89,M-SC,89.36,62971.0,-30839.0,107454.0,2.39,-32.87,170.64,81.67,203.0,-0.29,0.51,21.95,XR,NTT,CERAMICS
80,TATAMOTORS,771.32,1065.0,-51.41,M-LC,72.34,214945.0,-11052.0,97091.0,2.14,-4.89,45.17,38.07,68.0,-0.11,1.73,26.54,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,40.53,M-SC,75.53,145920.0,8342.0,86064.0,-3.10,6.06,58.98,68.62,182.0,0.10,1.17,26.50,X40N,NTT,TEXTILES
75,STARHEALTH,551.56,761.00,1.49,M-SC,90.43,166127.0,-34089.0,110109.0,-1.17,-17.03,66.28,37.97,194.0,-0.31,1.34,33.39,XY24,NTT,INSURANCE
38,INDIAMART,2327.09,4911.36,-52.40,H-SC,50.00,124184.0,848.0,136118.0,-1.06,0.69,109.61,111.05,108.0,0.01,1.00,21.68,AR,ATH,MISC
16,CAMPUS,294.86,393.00,NaN,NaN,85.11,154660.0,-13115.0,68963.0,-0.69,-7.82,44.59,33.28,175.0,-0.19,1.24,19.74,XY24,NTT,FOOTWEAR
89,VBL,500.38,672.85,-13.96,H-LC,1.06,148230.0,-4386.0,56994.0,-0.56,-2.87,38.45,34.47,28.0,-0.08,1.19,11.67,AR,ATH,FMCG


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1432.81,2275.00,-12.42,X-LC,54.26,218591.0,19430.0,97623.0,0.50,9.76,44.66,58.78,1.0,0.20,1.76,14.13,X40,BTT,IT
81,TCS,3888.13,4998.00,-15.52,X-LC,39.36,211872.0,-21416.0,88012.0,0.47,-9.18,41.54,28.55,2.0,-0.24,1.71,9.27,X40,BTT,IT
50,LTIM,5664.39,7197.33,-5.11,X-LC,88.30,152250.0,-17682.0,63671.0,0.58,-10.41,41.82,27.06,4.0,-0.28,1.23,26.22,X200,ATH,IT
72,SIEMENS,4406.90,7969.85,-8.81,X-LC,87.23,82578.0,-27594.0,116666.0,0.79,-25.05,141.28,80.85,5.0,-0.24,0.66,21.62,AR,ATH,ELECTRICAL
32,HAVELLS,1595.53,2069.16,-4.76,X-LC,10.64,183164.0,-1917.0,56854.0,0.34,-1.04,31.04,29.68,6.0,-0.03,1.47,12.65,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,4.18,H-SC,100.00,120495.0,22848.0,24822.0,1.77,23.40,20.60,48.82,138.0,0.92,0.97,33.86,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,91.49,100004.0,21642.0,12390.0,0.94,27.62,12.39,43.43,71.0,1.75,0.80,33.75,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.70,H-LC,46.81,100710.0,16907.0,7090.0,-0.02,20.17,7.04,28.63,42.0,2.38,0.81,34.58,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-17.01,X-LC,23.40,91953.0,16831.0,15329.0,1.10,22.41,16.67,42.81,11.0,1.10,0.74,15.32,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,40.53,M-SC,75.53,145920.0,8342.0,86064.0,-3.10,6.06,58.98,68.62,182.0,0.10,1.17,26.50,X40N,NTT,TEXTILES
57,POLYCAB,5005.95,7541.35,6.49,H-MC,56.38,101074.0,15973.0,27128.0,1.05,18.77,26.84,50.65,92.0,0.59,0.81,27.05,X40N,ATH,CABLES
55,PFIZER,4245.50,6318.30,4.18,H-SC,100.00,120495.0,22848.0,24822.0,1.77,23.40,20.60,48.82,138.0,0.92,0.97,33.86,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,2.11,H-SC,70.21,150628.0,12768.0,24989.0,1.21,9.26,16.59,27.39,118.0,0.51,1.21,30.55,X40,ATH,FMCG
33,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,86.17,153335.0,2422.0,58037.0,1.68,1.60,37.85,40.06,89.0,0.04,1.23,25.42,AR,ATH,AUTO


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-2.22,H-SC,15.96,109866.0,7862.0,37157.0,1.01,7.71,33.82,44.14,111.0,0.21,0.88,25.20,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,18.09,134216.0,5064.0,82006.0,0.75,3.92,61.10,67.42,155.0,0.06,1.08,30.54,XR,NTT,BANKS
33,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,86.17,153335.0,2422.0,58037.0,1.68,1.60,37.85,40.06,89.0,0.04,1.23,25.42,AR,ATH,AUTO
93,WIPRO,243.46,420.00,-10.62,M-LC,45.74,153363.0,2418.0,107032.0,0.58,1.60,69.79,72.51,69.0,0.02,1.23,15.47,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.40,H-SC,50.00,124184.0,848.0,136118.0,-1.06,0.69,109.61,111.05,108.0,0.01,1.00,21.68,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-55.73,L-SC,67.02,64358.0,-49191.0,89239.0,-0.06,-43.32,138.66,35.27,238.0,-0.55,0.52,57.52,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-23.67,H-SC,53.19,86925.0,-11055.0,32927.0,0.26,-11.28,37.88,22.32,130.0,-0.34,0.70,26.07,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-330.82,H-SC,80.85,98033.0,-35253.0,119845.0,0.00,-26.45,122.25,63.47,131.0,-0.29,0.79,64.33,XR,NTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,91.49,100004.0,21642.0,12390.0,0.94,27.62,12.39,43.43,71.0,1.75,0.80,33.75,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.70,H-LC,46.81,100710.0,16907.0,7090.0,-0.02,20.17,7.04,28.63,42.0,2.38,0.81,34.58,AR,ATH,INSURANCE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,4.18,H-SC,100.00,120495.0,22848.0,24822.0,1.77,23.40,20.60,48.82,138.0,0.92,0.97,33.86,X40,ATH,PHARMA
78,TANLA,917.30,1963.11,-38.20,H-SC,98.94,151524.0,-87891.0,360854.0,1.82,-36.71,238.15,114.01,110.0,-0.24,1.22,38.97,AR,ATH,IT
74,SONACOMS,578.05,1006.00,-25.14,M-MC,97.87,96792.0,-4367.0,79253.0,2.94,-4.32,81.88,74.03,151.0,-0.06,0.78,36.03,AR,BTT,AUTO
18,CERA,8421.60,10420.45,-19.09,X-SC,96.81,83434.0,-26047.0,52029.0,0.82,-23.79,62.36,23.73,64.0,-0.50,0.67,26.28,X40N,ATH,CERAMICS
24,DLF,694.61,1031.70,-15.67,H-LC,95.74,194902.0,20555.0,64064.0,0.09,11.79,32.87,48.53,30.0,0.32,1.57,27.70,AR,ATH,REALTY


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.57
1,25,39.04
2,50,68.65


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    33.15
X40N    13.99
XY25    13.97
X40     13.81
AR      12.16
XR      10.48
X200     1.23
SR       1.17
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.84
M-SC    18.53
X-LC    16.08
H-MC     8.96
M-LC     7.97
H-LC     6.70
X-MC     5.79
M-MC     5.77
X-SC     2.90
L-SC     2.40
L-LC     1.20
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,12.41,-0.91,33.45
BANKS,9.85,-14.41,67.01
IT,9.49,-19.26,85.37
MISC,6.67,-30.17,87.75
FINANCE,6.21,-23.86,82.67
INSURANCE,6.18,2.22,26.93
PAINTS,5.62,-17.93,52.83
AUTO,4.67,-7.64,54.67
ELECTRICAL,4.44,-4.80,58.12


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2851940.0,26
X40N,1206951.0,17
AR,1203906.0,13
XR,1147915.0,14
X40,712344.0,11
XY25,461249.0,10
SR,200309.0,2
X200,63671.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2302708.0,21
M-SC,2012193.0,21
X-LC,697096.0,12
M-MC,550923.0,5
H-MC,513656.0,8
X-SC,347927.0,4
X-MC,315540.0,5
M-LC,313971.0,6
L-SC,298971.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      914184.0      6
M-SC       XY24      745089.0      6
H-SC       AR        556630.0      3
           X40N      502121.0      6
M-SC       X40N      463400.0      6
M-MC       XY24      389664.0      3
X-LC       X40       388227.0      5
M-SC       XR        293073.0      4
H-SC       XR        279962.0      4
M-SC       X40       214044.0      2
H-LC       AR        213311.0      4
M-LC       XY24      189276.0      4
H-MC       XY24      180477.0      2
L-SC       XR        153596.0      2
X-LC       AR        139519.0      2
X-MC       XY24      139214.0      2
M-SC       XY25      119043.0      1
X-SC       XR        117942.0      1
           SR        116032.0      1
H-MC       XR        114304.0      1
X-SC       X40N      113953.0      2
M-LC       XR        107032.0      1
X-LC       XY25      105679.0      4
X-MC       X40N      100349.0      2
M-SC       AR         93267.0      1
           SR         84277.0      1
M-MC       XR         82006.0      1
L-SC       XY24       81486.0      1
M-MC       AR         79253.0      1
X-MC       XY25       75977.0      1
H-MC       XY25       73448.0      1
L-SC       AR         63889.0      1
X-LC       X200       63671.0      1
H-MC       X40        60262.0      2
           AR         58037.0      1
H-SC       X40        49811.0      2
L-LC       XY25       40121.0      1
H-LC       XY25       29318.0      1
H-MC       X40N       27128.0      1
M-LC       XY25       17663.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
